# Segmenting and Clustering Neighbourhoods in Toronto

## About

This notebook was made for the IBM Capstone Project.

#### Import dependencies

In [1]:
#!pip install bs4
#!pip install requests
#!pip install html5lib

In [2]:
# import libraries
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import numpy as np # library to handle data in a vectorized manner
from bs4 import BeautifulSoup # this module helps in web scrapping
import requests  # this module helps us to download a web page
import html5lib
import re  # for splitting using regex

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import pgeocode

import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## Part 1. Use Webscraping to create DataFrame on Toronto's postcodes, boroughs and neighbourhoods

#### Use Webscraping to extract Toronto neighbourhood Data

In [3]:
# this is the webpage url to extract the data from
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [4]:
# get the html data from the page
html_data = requests.get(url).text 

In [5]:
# convert all the html to soup
soup = BeautifulSoup(html_data,"html5lib")

In [6]:
# check html
#soup

In [7]:
postcode_data = pd.DataFrame(columns=["col0", "col1", "col2", "col3", "col4", "col5", "col6", "col7", "col8"])

for row in soup.find("tbody").find_all("tr"):
    col = row.find_all("td")
    col0 = col[0].text
    col1 = col[1].text
    col2 = col[2].text
    col3 = col[3].text
    col4 = col[4].text
    col5 = col[5].text
    col6 = col[6].text
    col7 = col[7].text
    col8 = col[8].text
    
    postcode_data = postcode_data.append({"col0":col0, "col1":col1, "col2":col2, "col3":col3, "col4":col4, "col5":col5, "col6":col6,"col7":col7, "col8":col8}, ignore_index=True)

In [8]:
num_rows, num_cols = postcode_data.shape
num_cols

9

In [9]:
postcode_data

,col0,col1,col2,col3,col4,col5,col6,col7,col8
0,\nM1ANot assigned\n\n,\nM2ANot assigned\n\n,\nM3ANorth York(Parkwoods)\n\n,\nM4ANorth York(Victoria Village)\n\n,\nM5ADowntown Toronto(Regent Park / Harbourfro...,\nM6ANorth York(Lawrence Manor / Lawrence Heig...,\nM7AQueen's Park(Ontario Provincial Governmen...,\nM8ANot assigned\n\n,\nM9AEtobicoke(Islington Avenue)\n\n
1,\nM1BScarborough(Malvern / Rouge)\n\n,\nM2BNot assigned\n\n,\nM3BNorth York(Don Mills)North\n\n,\nM4BEast York(Parkview Hill / Woodbine Garden...,"\nM5BDowntown Toronto(Garden District, Ryerson...",\nM6BNorth York(Glencairn)\n\n,\nM7BNot assigned\n\n,\nM8BNot assigned\n\n,\nM9BEtobicoke(West Deane Park / Princess Gard...
2,\nM1CScarborough(Rouge Hill / Port Union / Hig...,\nM2CNot assigned\n\n,\nM3CNorth York(Don Mills)South(Flemingdon Par...,\nM4CEast York(Woodbine Heights)\n\n,\nM5CDowntown Toronto(St. James Town)\n\n,\nM6CYork(Humewood-Cedarvale)\n\n,\nM7CNot assigned\n\n,\nM8CNot assigned\n\n,\nM9CEtobicoke(Eringate / Bloordale Gardens / ...
3,\nM1EScarborough(Guildwood / Morningside / Wes...,\nM2ENot assigned\n\n,\nM3ENot assigned\n\n,\nM4EEast Toronto(The Beaches)\n\n,\nM5EDowntown Toronto(Berczy Park)\n\n,\nM6EYork(Caledonia-Fairbanks)\n\n,\nM7ENot assigned\n\n,\nM8ENot assigned\n\n,\nM9ENot assigned\n\n
4,\nM1GScarborough(Woburn)\n\n,\nM2GNot assigned\n\n,\nM3GNot assigned\n\n,\nM4GEast York(Leaside)\n\n,\nM5GDowntown Toronto(Central Bay Street)\n\n,\nM6GDowntown Toronto(Christie)\n\n,\nM7GNot assigned\n\n,\nM8GNot assigned\n\n,\nM9GNot assigned\n\n
5,\nM1HScarborough(Cedarbrae)\n\n,\nM2HNorth York(Hillcrest Village)\n\n,\nM3HNorth York(Bathurst Manor / Wilson Height...,\nM4HEast York(Thorncliffe Park)\n\n,\nM5HDowntown Toronto(Richmond / Adelaide / Ki...,\nM6HWest Toronto(Dufferin / Dovercourt Villag...,\nM7HNot assigned\n\n,\nM8HNot assigned\n\n,\nM9HNot assigned\n\n
6,\nM1JScarborough(Scarborough Village)\n\n,\nM2JNorth York(Fairview / Henry Farm / Oriole...,\nM3JNorth York(Northwood Park / York Universi...,\nM4JEast YorkEast Toronto(The Danforth East)...,\nM5JDowntown Toronto(Harbourfront East / Unio...,\nM6JWest Toronto(Little Portugal / Trinity)\n\n,\nM7JNot assigned\n\n,\nM8JNot assigned\n\n,\nM9JNot assigned\n\n
7,\nM1KScarborough(Kennedy Park / Ionview / East...,\nM2KNorth York(Bayview Village)\n\n,\nM3KNorth York(Downsview)East (CFB Toronto)\n\n,\nM4KEast Toronto(The Danforth West / Riverdal...,\nM5KDowntown Toronto(Toronto Dominion Centre ...,\nM6KWest Toronto(Brockton / Parkdale Village ...,\nM7KNot assigned\n\n,\nM8KNot assigned\n\n,\nM9KNot assigned\n\n
8,\nM1LScarborough(Golden Mile / Clairlea / Oakr...,\nM2LNorth York(York Mills / Silver Hills)\n\n,\nM3LNorth York(Downsview)West\n\n,\nM4LEast Toronto(India Bazaar / The Beaches W...,\nM5LDowntown Toronto(Commerce Court / Victori...,\nM6LNorth York(North Park / Maple Leaf Park /...,\nM7LNot assigned\n\n,\nM8LNot assigned\n\n,\nM9LNorth York(Humber Summit)\n\n
9,\nM1MScarborough(Cliffside / Cliffcrest / Scar...,\nM2MNorth York(Willowdale / Newtonbrook)\n\n,\nM3MNorth York(Downsview)Central\n\n,\nM4MEast Toronto(Studio District)\n\n,\nM5MNorth York(Bedford Park / Lawrence Manor ...,\nM6MYork(Del Ray / Mount Dennis / Keelsdale a...,\nM7MNot assigned\n\n,\nM8MNot assigned\n\n,\nM9MNorth York(Humberlea / Emery)\n\n


In [10]:
# declare variables
cell_value = ''
temp_postcode = ''
temp_borough = ''
part_before_backet_substring = ''
temp_postcodes = []
temp_boroughs = []
temp_neighbourhoods = []
split_point = 3  # num characters for postcode

# loop through each col
for col in range(0, num_cols):
    
    col_list = postcode_data[postcode_data.columns[col]]
    
    #col_list.map(lambda x: x.lstrip('\n').rstrip('\n\n'))
    
    # for each row in this column
    for row in range(0, num_rows):
        
        cell_value = col_list[row]
        # check to see if this postcode is assigned a neighbourhood, otherwise we don't want it
        if (cell_value.find('Not assigned')==-1):
            
            #if (cell_value.find('\n')==-1):
            cell_value = cell_value.replace('\n', ' ').strip()
            
            # get the part of the string before the bracket
            part_before_backet_substring = cell_value.split('(')[0]

            # get postcode from first 3 characters, get borough as the rest of the string
            temp_postcode, temp_borough = part_before_backet_substring[:split_point], part_before_backet_substring[split_point:]
            
            # check boroughs for boroughs that don't fit the pattern & fix them
            #if (temp_borough.find('East YorkEast Toronto')!=-1):
            #    temp_borough = temp_borough[:9] + '/' + temp_borough[9:]
            
            # get the substring inside the brackets
            bracket_substring = cell_value[cell_value.find("(")+1:cell_value.find(")")]
            # replace all '/' with ',' instead
            bracket_substring = bracket_substring.replace(' / ', ', ')

            temp_postcodes.append(temp_postcode)
            temp_boroughs.append(temp_borough)
            temp_neighbourhoods.append(bracket_substring)
                            

In [11]:
# merge the 3 lists into a single DataFrame
postcode_df = pd.DataFrame({'PostalCode': temp_postcodes, 'Borough': temp_boroughs, 'Neighbourhood': temp_neighbourhoods}, columns=['PostalCode', 'Borough', 'Neighbourhood'])

In [12]:
# fix up neighbourhood values that don't fit the usual pattern
postcode_df['Borough'] = postcode_df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                                         'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                                         'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                                         'MississaugaCanada Post Gateway Processing Centre':'Mississauga','The Danforth  East':'The Danforth East'})

In [13]:
postcode_df.head(15)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


<p style="color:red">The dataset has 103 rows...

In [14]:
postcode_df.shape

(103, 3)

## Part 2. Get co-ordinates for the postcodes and add them to the DataFrame

In [15]:
temp_lats = []
temp_longs = []

# loop through postcode list looking up lat & log for each
for postal_code in temp_postcodes:
    nomi = pgeocode.Nominatim('ca')
    location = nomi.query_postal_code(postal_code)
    #print(location.latitude, location.longitude)
    # save lat & long to lists
    temp_lats.append(location.latitude)
    temp_longs.append(location.longitude)

In [16]:
# merge the 5 lists into a single DataFrame
df = pd.DataFrame({'PostalCode': temp_postcodes, 'Borough': temp_boroughs, 'Neighbourhood': temp_neighbourhoods, 'Latitude': temp_lats, 'Longitude': temp_longs}, columns=['PostalCode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude'])

In [17]:
df['Borough'] = df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                       'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                       'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                       'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [18]:
# check the new DataFrame with the co-ordinates
df.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.8113,-79.1930
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.7878,-79.1564
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7678,-79.1866
3,M1G,Scarborough,Woburn,43.7712,-79.2144
4,M1H,Scarborough,Cedarbrae,43.7686,-79.2389


In [19]:
# check how many postcodes are missing values
df.isna().sum()

PostalCode       0
Borough          0
Neighbourhood    0
Latitude         1
Longitude        1
dtype: int64

In [20]:
# drop rows if no co-ordinates can be looked up for that postcode
df = df.dropna(subset = ["Latitude"], inplace=False).reset_index(drop=True)

In [21]:
# check for missing values now
df.isna().sum()

PostalCode       0
Borough          0
Neighbourhood    0
Latitude         0
Longitude        0
dtype: int64

In [22]:
# use updated DataFrame to updated lists to use later
temp_postcodes = df['PostalCode']
temp_boroughs = df['Borough']
temp_neighbourhoods = df['Neighbourhood']
temp_lats = df['Latitude']
temp_longs = df['Longitude']

<p style="color:red">For 2nd part, the final DataFrame with co-ordinates looks like this...

<p style="color:red">The co-ordinates may be slightly different to those provided in the assignment, because I used a different library (pgeocode) to find the co-ordinates as the suggested Geocoder library didn't work

<p style="color:red">Also, the postcodes are in a different order (i.e. each postcode has a different index id) to the marking guide because I wrote my own code to extract the data

In [23]:
df.head(30)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.8113,-79.1930
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.7878,-79.1564
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7678,-79.1866
3,M1G,Scarborough,Woburn,43.7712,-79.2144
4,M1H,Scarborough,Cedarbrae,43.7686,-79.2389
5,M1J,Scarborough,Scarborough Village,43.7464,-79.2323
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.7298,-79.2639
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.7122,-79.2843
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.7247,-79.2312
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.6952,-79.2646


## Part 3. Visualize Toronto neighborhoods and show how they cluster together

#### Create a map of Toronto with neighbourhoods superimposed on top

In [24]:
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [25]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for postcode, lat, lng, borough, neighbourhood in zip(df['PostalCode'], df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {} {}'.format(neighbourhood, borough, postcode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Use Foursquare API to get the top venues for each neighbourhood

In [26]:
CLIENT_ID = 'QX5S2PBXBBYJQFFFCTJUCYY0WCZC52WJGP4Z04LPWMH5PZ03' # your Foursquare ID
CLIENT_SECRET = 'WJMHCJY25OQOG1F34DZE1Y0RJWMGOH5EP5E052BYU5JESYGR' # your Foursquare Secret
ACCESS_TOKEN = 'RFLADLE0DBAIPYKWSZH0LFEKGIAGGMQSGLR5LKHEP203BUIG' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QX5S2PBXBBYJQFFFCTJUCYY0WCZC52WJGP4Z04LPWMH5PZ03
CLIENT_SECRET:WJMHCJY25OQOG1F34DZE1Y0RJWMGOH5EP5E052BYU5JESYGR


In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [28]:
toronto_venues = getNearbyVenues(temp_neighbourhoods, temp_lats, temp_longs)

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale
York Mills West
Willowdale
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
The Danforth  East
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawr

In [29]:
# check the size of the resulting dataframe
print(toronto_venues.shape)
toronto_venues.head()

(2155, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.8113,-79.1930,Canadian Appliance Source Whitby,43.808353,-79.191331,Home Service
1,"Rouge Hill, Port Union, Highland Creek",43.7878,-79.1564,Fox and Fiddle,43.789082,-79.154459,Bar
2,"Guildwood, Morningside, West Hill",43.7678,-79.1866,Chick-N-Joy,43.768752,-79.187982,Fried Chicken Joint
3,"Guildwood, Morningside, West Hill",43.7678,-79.1866,Little Caesars Pizza,43.769046,-79.184386,Pizza Place
4,"Guildwood, Morningside, West Hill",43.7678,-79.1866,LCBO,43.771462,-79.184384,Liquor Store


In [30]:
# check how many venues were returned for each neighbourhood
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood, Long Branch",8,8,8,8,8,8
"Bathurst Manor, Wilson Heights, Downsview North",7,7,7,7,7,7
Bayview Village,5,5,5,5,5,5
"Bedford Park, Lawrence Manor East",26,26,26,26,26,26
Berczy Park,93,93,93,93,93,93
"Birch Cliff, Cliffside West",4,4,4,4,4,4
"Brockton, Parkdale Village, Exhibition Place",39,39,39,39,39,39
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",58,58,58,58,58,58


In [31]:
# find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 255 uniques categories.


### Analyze each neighbourhood

In [32]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighbourhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Cafeteria,College Gym,College Rec Center,College Stadium,College Theater,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Knitting Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Newsagent,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [33]:
# get new DataFrame size
toronto_onehot.shape

(2155, 256)

In [34]:
# group rows by neighbourhood and by taking the mean of the frequency of occurrence of each category
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Cafeteria,College Gym,College Rec Center,College Stadium,College Theater,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Knitting Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Newsagent,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000

In [35]:
# get size of new DataFrame
toronto_grouped.shape

(96, 256)

In [36]:
# print each neighbourhood along with the top 5 most common venues
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0  Latin American Restaurant   0.2
1               Skating Rink   0.2
2            Badminton Court   0.2
3             Breakfast Spot   0.2
4                  Newsagent   0.2


----Alderwood, Long Branch----
               venue  freq
0        Pizza Place  0.12
1  Convenience Store  0.12
2     Sandwich Place  0.12
3        Coffee Shop  0.12
4                Pub  0.12


----Bathurst Manor, Wilson Heights, Downsview North----
                       venue  freq
0  Middle Eastern Restaurant  0.14
1   Mediterranean Restaurant  0.14
2              Deli / Bodega  0.14
3          Other Repair Shop  0.14
4                Coffee Shop  0.14


----Bayview Village----
         venue  freq
0      Dog Run   0.2
1        Trail   0.2
2  Gas Station   0.2
3         Park   0.2
4  Flower Shop   0.2


----Bedford Park, Lawrence Manor East----
                venue  freq
0         Pizza Place  0.08
1      Sandwich Place  0.08
2         Coffee Shop  0.08
3

### Put top venues for each neighbourhood into a pandas dataframe

In [37]:
# function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [38]:
# create the new dataframe and display the top 10 venues for each neighbourhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.tail(50)

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
46,"Lawrence Manor, Lawrence Heights",Clothing Store,Coffee Shop,Restaurant,Cosmetics Shop,Women's Store,Furniture / Home Store,Sandwich Place,Food Court,Men's Store,Sushi Restaurant
47,Lawrence Park,Photography Studio,Park,Accessories Store,New American Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant
48,Leaside,Department Store,Sporting Goods Shop,Electronics Store,Sports Bar,Restaurant,Bank,Coffee Shop,Bagel Shop,Breakfast Spot,Liquor Store
49,"Little Portugal, Trinity",Bar,Cocktail Bar,Coffee Shop,Restaurant,Asian Restaurant,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Yoga Studio,Skating Rink,Seafood Restaurant
50,"Malvern, Rouge",Home Service,Accessories Store,Newsagent,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater
51,"Milliken, Agincourt North, Steeles East, L'Amo...",Intersection,Pharmacy,Accessories Store,Movie Theater,Neighborhood,Music Venue,Music Store,Museum,Moving Target,Monument / Landmark
52,"Mimico NW, The Queensway West, South of Bloor,...",Burrito Place,Gym / Fitness Center,Sandwich Place,Fish & Chips Shop,Food & Drink Shop,Middle Eastern Restaurant,Burger Joint,Sushi Restaurant,Liquor Store,Thai Restaurant
53,"Moore Park, Summerhill East",Gym,Thai Restaurant,Park,Grocery Store,Moving Target,Neighborhood,Music Venue,Music Store,Museum,Moroccan Restaurant
54,"New Toronto, Mimico South, Humber Bay Shores",Intersection,Accessories Store,Newsagent,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater
55,"North Park, Maple Leaf Park, Upwood Park",Basketball Court,Bakery,Massage Studio,Moving Target,New American Restaurant,Neighborhood,Music Venue,Music Store,Museum,Accessories Store


###  Cluster neighbourhoods using K-means

In [39]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 2, 0, 0, 0, 0, 0, 2])

In [40]:
toronto_grouped_clustering.head()

,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Cafeteria,College Gym,College Rec Center,College Stadium,College Theater,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Knitting Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Newsagent,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

#### Create a new dataframe that includes the cluster as well as the top 10 venues for each neighbourhood

In [41]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighbourhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), how='right', on='Neighbourhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,M1S,Scarborough,Agincourt,43.7946,-79.2644,0,Latin American Restaurant,Skating Rink,Badminton Court,Breakfast Spot,Newsagent,Neighborhood,Music Venue,Music Store,Museum,Movie Theater
88,M8W,Etobicoke,"Alderwood, Long Branch",43.6021,-79.5402,0,Pizza Place,Convenience Store,Sandwich Place,Coffee Shop,Pub,Gym,Dance Studio,Pharmacy,Newsagent,Modern European Restaurant
28,M3H,North York,"Bathurst Manor, Wilson Heights, Downsview North",43.7535,-79.4472,0,Middle Eastern Restaurant,Mediterranean Restaurant,Deli / Bodega,Other Repair Shop,Coffee Shop,Fried Chicken Joint,Pizza Place,Jewelry Store,Noodle House,Monument / Landmark
19,M2K,North York,Bayview Village,43.7797,-79.3813,2,Dog Run,Trail,Gas Station,Park,Flower Shop,Movie Theater,Music Venue,Music Store,Museum,Moving Target
62,M5M,North York,"Bedford Park, Lawrence Manor East",43.7335,-79.4177,0,Pizza Place,Sandwich Place,Coffee Shop,Italian Restaurant,Thai Restaurant,Pub,Greek Restaurant,Grocery Store,Comfort Food Restaurant,Fast Food Restaurant


### Visualise the clusters on the map

In [42]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

#### Cluster 1

<p style="color:red">These neighbourhoods seem have these in common: Home Services, ATMs, Moroccan Restaurants

In [43]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Scarborough,0,Latin American Restaurant,Skating Rink,Badminton Court,Breakfast Spot,Newsagent,Neighborhood,Music Venue,Music Store,Museum,Movie Theater
88,Etobicoke,0,Pizza Place,Convenience Store,Sandwich Place,Coffee Shop,Pub,Gym,Dance Studio,Pharmacy,Newsagent,Modern European Restaurant
28,North York,0,Middle Eastern Restaurant,Mediterranean Restaurant,Deli / Bodega,Other Repair Shop,Coffee Shop,Fried Chicken Joint,Pizza Place,Jewelry Store,Noodle House,Monument / Landmark
62,North York,0,Pizza Place,Sandwich Place,Coffee Shop,Italian Restaurant,Thai Restaurant,Pub,Greek Restaurant,Grocery Store,Comfort Food Restaurant,Fast Food Restaurant
56,Downtown Toronto,0,Coffee Shop,Bakery,Hotel,Seafood Restaurant,Café,Beer Bar,Japanese Restaurant,Restaurant,Cocktail Bar,Pharmacy
9,Scarborough,0,Skating Rink,General Entertainment,College Stadium,Café,Neighborhood,Music Venue,Music Store,Museum,Moving Target,Accessories Store
78,West Toronto,0,Café,Coffee Shop,Gift Shop,Breakfast Spot,Bar,Thrift / Vintage Store,Brewery,Hawaiian Restaurant,Mexican Restaurant,Gym
68,Downtown Toronto,0,Italian Restaurant,Coffee Shop,Café,Bar,Bank,Grocery Store,Park,Bakery,French Restaurant,Restaurant
4,Scarborough,0,Lounge,Gaming Cafe,Trail,Accessories Store,Music Venue,Music Store,Museum,Moving Target,Movie Theater,Moroccan Restaurant
57,Downtown Toronto,0,Coffee Shop,Bubble Tea Shop,Japanese Restaurant,Italian Restaurant,Café,Clothing Store,Middle Eastern Restaurant,Restaurant,Sandwich Place,Comic Shop


#### Cluster 2

<p style="color:red">These neighbourhoods seem have a lot of parks, coffee shops, pizza places

In [44]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
82,West Toronto,1,Park,Accessories Store,Newsagent,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater
17,North York,1,Park,Accessories Store,Newsagent,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater
44,Central Toronto,1,Photography Studio,Park,Accessories Store,New American Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant
25,North York,1,Food & Drink Shop,Park,Accessories Store,New American Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant
40,East York/East Toronto,1,Park,Convenience Store,Moroccan Restaurant,Neighborhood,Music Venue,Music Store,Museum,Moving Target,Movie Theater,Accessories Store
97,York,1,Jewelry Store,Park,Moroccan Restaurant,Neighborhood,Music Venue,Music Store,Museum,Moving Target,Movie Theater,Accessories Store
23,North York,1,Park,Convenience Store,Construction & Landscaping,Accessories Store,New American Restaurant,Neighborhood,Music Venue,Music Store,Museum,Movie Theater


#### Cluster 3

<p style="color:red">These neighbourhoods seem have a lot of parks, ATMs and either music venues/stores and or restaurants

In [45]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,North York,2,Dog Run,Trail,Gas Station,Park,Flower Shop,Movie Theater,Music Venue,Music Store,Museum,Moving Target
74,York,2,Park,Mexican Restaurant,Sporting Goods Shop,Bakery,Beer Store,Gym,Women's Store,Modern European Restaurant,Mobile Phone Shop,Molecular Gastronomy Restaurant
26,North York,2,Park,Electronics Store,River,Pool,Home Service,Trail,Accessories Store,Moving Target,Music Venue,Music Store
27,North York,2,Park,Electronics Store,River,Pool,Home Service,Trail,Accessories Store,Moving Target,Music Venue,Music Store
64,Central Toronto,2,Home Service,Trail,Park,Accessories Store,Moroccan Restaurant,Music Venue,Music Store,Museum,Moving Target,Movie Theater
59,Downtown Toronto,2,Music Venue,Harbor / Marina,Café,Park,Accessories Store,Moving Target,Neighborhood,Music Store,Museum,Moroccan Restaurant
73,York,2,Grocery Store,Deli / Bodega,Playground,Park,Hockey Arena,Field,Tennis Court,Trail,Museum,Neighborhood
48,Central Toronto,2,Gym,Thai Restaurant,Park,Grocery Store,Moving Target,Neighborhood,Music Venue,Music Store,Museum,Moroccan Restaurant
46,Central Toronto,2,Playground,Gym Pool,Garden,Park,Movie Theater,Music Venue,Music Store,Museum,Moving Target,Monument / Landmark
90,Etobicoke,2,Flower Shop,Park,Baseball Field,Accessories Store,New American Restaurant,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant


#### Cluster 4

<p style="color:red">This neighbourhood has a lot of auto garages, ATMs, and movie theaters

In [46]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Scarborough,3,Intersection,Pharmacy,Accessories Store,Movie Theater,Neighborhood,Music Venue,Music Store,Museum,Moving Target,Monument / Landmark
87,Etobicoke,3,Intersection,Accessories Store,Newsagent,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater


#### Cluster 5

<p style="color:red">This neighbourhood has a lot of Korean or Mexican restaurants

In [47]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,4,Home Service,Accessories Store,Newsagent,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater
63,Central Toronto,4,Home Service,Accessories Store,Newsagent,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater
